In [1]:
import pandas as pd
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

## 추천받은 포트폴리오 -> 잔고(detail 테이블)

In [2]:
instruments_m = pd.read_pickle('../../data/processed/instruments_m.pkl')
price_db = pd.read_pickle('../../data/external/price_db_d.pkl')
advised_pf = pd.read_pickle('../../data/processed/advised_portfolios.pkl')    

In [3]:
userid = 'A00'
username = '투자자0'
current_date = '2021-02-01'
risk_profile = 2

# c_date: 추천 포트폴리오DB에서 사용자가 입력한 날짜와 가장 가까운 날짜.
c_date = advised_pf.loc[advised_pf.date <= current_date, ['date']].max().date
new_units = {'A122260': 2, 'A114260': 4, 'A130730': 2, 'A329650': 13, 'A266370': 4}
prices = {'A122260': [101185.0, 'KRW'], 'A114260': [57900.0, 'KRW'], 'A130730': [100975.0, 'KRW'], 'A329650': [10980.0, 'KRW'], 'A266370': [21325.0, 'KRW']}
df = advised_pf.loc[(advised_pf.date==c_date) & (advised_pf.risk_profile==risk_profile), :]
remaining_cash = 135786

In [6]:
# 보유수량
detail = pd.DataFrame.from_dict(new_units, orient='index').rename(columns={0:'quantity'})
detail.index.name='itemcode'

# 매입가격
temp = pd.DataFrame.from_dict(prices, orient='index').rename(columns={0:'cost_price'}).drop([1], axis=1)
temp.index.name='itemcode'

# 병합
detail = detail.merge(temp, left_index=True, right_index=True, how='inner')

detail['price'] = detail['cost_price']
detail['cost_value'] = detail['quantity']*detail['cost_price']
detail['value'] = detail['quantity']*detail['cost_price']  # 매입가와 평가가격 동일하다고 가정
detail = detail.merge(instruments_m.loc[:, ['itemcode', 'itemname', 'asset_class']], left_on='itemcode', right_on='itemcode', how='left')
detail = detail.reset_index(drop=True)

In [8]:
df_cash = {
    'itemcode': 'C000001',
    'quantity': remaining_cash,
    'cost_price': 1,
    'cost_value': remaining_cash,
    'value': remaining_cash,
    'itemname': '현금',
    'asset_class': 'Cash'
}
df_cash = pd.DataFrame.from_dict(df_cash, orient='index').T

In [9]:
detail = pd.concat([detail, df_cash])

In [10]:
detail['date'] = c_date
detail['date'] = pd.to_datetime(detail['date'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y 4:0:00 PM').astype(str)
detail['userid'] = userid
detail['username'] = username
detail['original'] = 'N'
detail['group_by'] = ''

In [11]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,
0,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,


In [14]:
detail['wt'] = detail.value.transform(lambda x: x/x.sum())

In [12]:
detail = detail.reset_index(drop=True)

In [13]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,
5,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,


## 리밸런싱

In [17]:
current_date

'2021-02-01'

In [26]:
end_date = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].max()

In [28]:
end_date

'2021-03-19'

In [35]:
dates = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].unique()

array(['2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05',
       '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11',
       '2021-02-12', '2021-02-15', '2021-02-16', '2021-02-17',
       '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23',
       '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01',
       '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05',
       '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11',
       '2021-03-12', '2021-03-15', '2021-03-16', '2021-03-17',
       '2021-03-18', '2021-03-19'], dtype=object)

In [44]:
dates[::5]

array(['2021-02-02', '2021-02-09', '2021-02-16', '2021-02-23',
       '2021-03-02', '2021-03-09', '2021-03-16'], dtype=object)

In [46]:
current_date

'2021-02-01'

In [48]:
from datetime import datetime

In [60]:
dt = datetime.strptime('2021-02-01', '%Y-%m-%d')

In [63]:
dt = str(dt.month)+'/'+str(dt.day)+'/'+str(dt.year)+' 09:00:00 AM'

In [64]:
dt

'2/1/2021 09:00:00 AM'

In [54]:
now = datetime.now()

In [55]:
now

datetime.datetime(2021, 3, 23, 23, 57, 11, 622959)